### Mais informações sobre cada indicador do IBGE

Os indicadores das pesquisas são hierarquicamente organizados, podendo chegar a 6 níveis (!). Para ler o json e extrair as relações certas entre cada indicador e seu significado, precisamos navegar no json e buscar:
* posição: nível da hierarquia, ex: 1.1.2.1.2
* indicador: grão daquela medida, ex: Pré-escola, 6º ano, Privado, Estadual
* classe: se é numérico, ex: I, N, T
* unidade: fala o que está sendo medido, ex: "docentes - N"

In [1]:
import json
import pandas as pd

### 0. Constantes

In [2]:
path_data = 'D:/data_dash_covid/datasets/'

### 1. Read data

In [3]:
with open(path_data + 'raw/ibge_cidades/indicadores-da-pesquisa-10096.json') as json_file:
    mydata = json.load(json_file)

### 2. Walk through nested json

Brilliant solution from [here](https://hackersandslackers.com/extract-data-from-complex-json-python/)

In [4]:
"""Extract nested values from a JSON tree."""


def json_extract(obj, key):
    """Recursively fetch values from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values

### 3. Get additional info

Para cada Pesquisa:
* posição
* indicador
* classe
* unidade

In [5]:
list_posicao = json_extract(mydata, 'posicao')
print(len(list_posicao))

1090


In [6]:
list_indicador = json_extract(mydata, 'indicador')
print(len(list_indicador))

1090


In [7]:
list_ids = json_extract(mydata, 'id')
list_ids = [i for i in list_ids if isinstance(i,int)]
print(len(list_ids))

1090


In [8]:
teste_df = list(zip(list_ids,list_posicao,list_indicador))
teste_df

[(84179, '1', 'Total'),
 (84180, '1.1', 'Situação operacional'),
 (84181,
  '1.1.1',
  'Aberta, mesmo que parcialmente, em produção ou funcionamento'),
 (84182, '1.1.1.1', 'Percentual em relação ao número de empresas'),
 (84183, '1.1.1.2', 'Impacto geral causado pela pandemia da COVID-19'),
 (84184, '1.1.1.2.1', 'Impacto negativo'),
 (84185, '1.1.1.2.2', 'Impacto pequeno ou inexistente'),
 (84186, '1.1.1.2.3', 'Impacto positivo'),
 (84187, '1.1.1.2.4', 'Não soube responder'),
 (84188,
  '1.1.1.3',
  'Impacto na venda de produtos ou serviços comercializados'),
 (84189, '1.1.1.3.1', 'Impacto negativo'),
 (84190, '1.1.1.3.2', 'Impacto pequeno ou inexistente'),
 (84191, '1.1.1.3.3', 'Impacto positivo'),
 (84192, '1.1.1.3.4', 'Não soube responder'),
 (84193,
  '1.1.1.4',
  'Impacto na fabricação de produtos ou atendimento aos clientes'),
 (84194, '1.1.1.4.1', 'Impacto negativo'),
 (84195, '1.1.1.4.2', 'Impacto pequeno ou inexistente'),
 (84196, '1.1.1.4.3', 'Impacto positivo'),
 (84197, '1.

In [9]:
# Cria DF com os detalhes de cada indicador

df = pd.DataFrame(data=teste_df)
df.columns = ['id_indicador','arvore_indicador','descri_indicador']

In [10]:
df['nivel_indicador'] = df['arvore_indicador'].str.count('\.') + 1

In [11]:
df['nivel_indicador'].unique()

array([1, 2, 3, 4, 5, 6, 7, 8], dtype=int64)

In [14]:
df

,id_indicador,arvore_indicador,descri_indicador,nivel_indicador
0,84179,1,Total,1
1,84180,1.1,Situação operacional,2
2,84181,1.1.1,"Aberta, mesmo que parcialmente, em produção ou...",3
3,84182,1.1.1.1,Percentual em relação ao número de empresas,4
4,84183,1.1.1.2,Impacto geral causado pela pandemia da COVID-19,4
...,...,...,...,...
1085,84174,3.4.6.1.3.1,Percentual em relação ao número de empresas,6
1086,84175,3.4.6.1.3.2,Fechamento devido à pandemia da COVID-19,6
1087,84176,3.4.6.1.3.2.1,Sim,7
1088,84177,3.4.6.1.3.2.2,Não,7


In [24]:
df[df['arvore_indicador']=='1.1.1.7.10']

,id_indicador,arvore_indicador,descri_indicador,nivel_indicador
43,84222,1.1.1.7.10,Não soube responder,5


### 4. Concat full name

* Concatena o nome do nível-pai no nome de cada indicador de nível-filho
* Cria um dicionário que ao final será mapeado para a coluna "nome_completo_indicador"

In [32]:
# Pega os primeiros conjuntos de números antes do último ponto final (ou seja, a árvore do pai)

s = '1.1.1.7.9' # "1.1.1.7.10", "1.1.1.7.9", "1.1.1.7"
s.rsplit('.', 1)[0]

'1.1.1.7'

In [33]:
def concat_parent_name_by_level(df):
    k = df['nivel_indicador'].max() + 1
    mydict = {}
    for i,row in df[df['nivel_indicador']==1].iterrows():
        mydict[row.arvore_indicador] = row.descri_indicador
    for j in range(2,k):
        for i,row in df[df['nivel_indicador']==j].iterrows():
            mydict[row.arvore_indicador] = mydict[row.arvore_indicador.rsplit('.', 1)[0]] + ' - ' + row.descri_indicador
        j += 1
    return mydict

In [35]:
# Adiciona coluna nome_completo_indicador

mydict = concat_parent_name_by_level(df)
df['nome_completo_indicador'] = df['arvore_indicador'].map(mydict)

In [36]:
df

,id_indicador,arvore_indicador,descri_indicador,nivel_indicador,nome_completo_indicador
0,84179,1,Total,1,Total
1,84180,1.1,Situação operacional,2,Total - Situação operacional
2,84181,1.1.1,"Aberta, mesmo que parcialmente, em produção ou...",3,"Total - Situação operacional - Aberta, mesmo q..."
3,84182,1.1.1.1,Percentual em relação ao número de empresas,4,"Total - Situação operacional - Aberta, mesmo q..."
4,84183,1.1.1.2,Impacto geral causado pela pandemia da COVID-19,4,"Total - Situação operacional - Aberta, mesmo q..."
...,...,...,...,...,...
1085,84174,3.4.6.1.3.1,Percentual em relação ao número de empresas,6,Atividade - Serviços - Outros serviços - Situa...
1086,84175,3.4.6.1.3.2,Fechamento devido à pandemia da COVID-19,6,Atividade - Serviços - Outros serviços - Situa...
1087,84176,3.4.6.1.3.2.1,Sim,7,Atividade - Serviços - Outros serviços - Situa...
1088,84177,3.4.6.1.3.2.2,Não,7,Atividade - Serviços - Outros serviços - Situa...


### 5. Export details ind

Para fazer um loop com todas as pesquisas, rodei no PyCharm

In [205]:
# n_pesquisa = 10096

In [210]:
# df.to_csv(path_data+'prepared/details_ind_ibge/details_ind_pesquisa_'+str(n_pesquisa)+'.csv', 
#           index=False, sep=';', encoding='utf-8')